In [11]:
from bs4 import BeautifulSoup as bs
import re
import requests
import pandas as pd
import time

In [12]:
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

In [13]:
data_all = pd.read_csv('./1090531_test_try_catch.csv') #2004~2020每場比賽
#data_all['隊伍'].value_counts()   #誠泰 5955 #米迪亞 1467 不管他們 #中信也不要， 共26769

In [14]:
#data_all

In [15]:
del data_all['Unnamed: 0'] 
#刪掉三隊
data_all = data_all[(~data_all['主隊'].isin(['中信','誠泰','米迪亞'])) & (~data_all['客隊'].isin(['中信','誠泰','米迪亞']))]
#刪掉投手與非先發
data_all = data_all[(data_all['守備位置'] != 'P')&(~data_all['守備位置'].str.startswith(' ('))]
#data_all['隊伍'].value_counts() 

In [16]:
#data_all['守備位置'].value_counts() 
data_all

,index,主/客,隊伍,先發名單,守備位置,球員ID,TeamID,年,月,日,場次,主隊,客隊,主投,客投,客投ID,主投ID,輸贏
0,0,主隊,兄弟,林 明 憲,LF,E0A4,E01,2004,2,28,1,兄弟,興農,風 神,陽 建 福,B0A6,T097,L
1,1,主隊,兄弟,馮 勝 賢,2B,E0B5,E01,2004,2,28,1,兄弟,興農,風 神,陽 建 福,B0A6,T097,L
2,2,主隊,兄弟,陳 致 遠,CF,E0C4,E01,2004,2,28,1,兄弟,興農,風 神,陽 建 福,B0A6,T097,L
3,3,主隊,兄弟,彭 政 閔,RF,E0C1,E01,2004,2,28,1,兄弟,興農,風 神,陽 建 福,B0A6,T097,L
4,4,主隊,兄弟,蔡 豐 安,1B,T067,E01,2004,2,28,1,兄弟,興農,風 神,陽 建 福,B0A6,T097,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128306,20,客隊,中信兄弟,蘇緯達,1B,H017,E02,2020,5,30,69,富邦,中信兄弟,伍鐸,萊福力,004569,E0L7,W
128307,21,客隊,中信兄弟,詹子賢,LF,k453,E02,2020,5,30,69,富邦,中信兄弟,伍鐸,萊福力,004569,E0L7,W
128308,22,客隊,中信兄弟,許基宏,DH,H230,E02,2020,5,30,69,富邦,中信兄弟,伍鐸,萊福力,004569,E0L7,W
128309,23,客隊,中信兄弟,黃鈞聲,C,R610,E02,2020,5,30,69,富邦,中信兄弟,伍鐸,萊福力,004569,E0L7,W


In [17]:
Lamigo = data_all[(data_all['隊伍'].isin(['Lamigo','樂天']))] #做lamigo
ID_lst = list(set(Lamigo['球員ID'].values))
print(len(set(Lamigo['球員ID'].values)))
type_ = 1 ######################打者習慣#################
df_total = pd.DataFrame()
#for i in range(len(H_Lamigo)):
num = 0
for i in ID_lst:
    print('現在到：'+ str(num))
    url = 'http://www.cpbl.com.tw/players/apart.html?player_id={}'.format(i)
    time.sleep(2)
    re = requests.get(url) 
    soup = bs(re.text, 'html.parser')
    year_lst = [option.text for option in soup.find_all("select",{"class":"date_select"})[1].find_all("option")]
    for year in year_lst:
        response = requests.get(url + f'&year={year}'+f'&type=0{type_}')
        print(url + f'&year={year}'+f'&type=0{type_}')
        if response.status_code != 200:
            print(print('URL發生錯誤：' + url))
        soup = bs(response.text, 'html.parser')
        table = soup.find_all('tr')
        if len(table)>3:
            #變數名稱
            title_list = table[2].text.split('\n')
            title_list[0:2] = ['num','R/L']
            df = pd.DataFrame(columns = title_list)
            for i in range(3,len(table)):
                df.loc[i] = table[i].text.split('\n')
            df['year'] = year
            df['ID'] = i
            df['team'] = soup.find('div','h1_side').a['href'].replace('/web/team_brief.php?&team=','')
            df_total = pd.concat([df_total,df])
    num +=1
df_total

94
現在到：0
http://www.cpbl.com.tw/players/apart.html?player_id=A050&year=2007&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=A050&year=2006&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=A050&year=2005&type=01
現在到：1
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2019&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2018&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2017&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2016&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2015&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2014&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2013&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2012&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2011&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R367&year=2010&type=01
http://www

ConnectionError: HTTPConnectionPool(host='www.cpbl.com.tw', port=80): Max retries exceeded with url: /players/apart.html?player_id=A011 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A4423E0D08>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

# 按到這停

In [ ]:
H_Lamigo = data_all[(data_all['隊伍'].isin(['Lamigo','樂天'])) & (data_all['主隊'].isin(['Lamigo','樂天']))] #看客投ID
# H_Lamigo['球員ID'].value_counts() #每個球員上場次數
A_Lamigo = data_all[(data_all['隊伍'].isin(['Lamigo','樂天'])) & (data_all['客隊'].isin(['Lamigo','樂天']))] #看主投ID

In [ ]:
H_Lamigo['球員ID'].iloc[0]

In [ ]:
H_Lamigo['球員ID']

In [ ]:
set(H_Lamigo['TeamID'].values)

In [ ]:
HA_player = pd.read_csv('./球員分項5表/HA_type2.csv')
HA_player['TP'].value_counts()
batter_HA = HA_player[HA_player['TP'] == 0]

In [ ]:
H_Lamigo['客隊'].value_counts()

In [ ]:
#左右投
H_Lamigo['客投ID'].value_counts()

In [ ]:
Away_pitcher_ID_list = list(set(H_Lamigo['客投ID']))
for i in Away_pitcher_ID_list:
    

In [ ]:
#左右投
base_detail = pd.read_csv('./基本資料/150_player.csv')
base_detail['ID'].values
pitcher['RorL'].loc[0][3]

In [ ]:
pitcher

In [ ]:
match = data_all[(data_all['年'] == 2004) & (data_all['月'] == 3) & (data_all['日'] == 24)]
match

In [ ]:
#用球員ID去配對HA
match['球員ID'].iloc[0]

In [ ]:
'R610' in batter_HA['ID'].values

In [ ]:
OPS = batter_HA.OBP[batter_HA['ID'].isin([match['球員ID'].iloc[0]]) & batter_HA['year'].isin([match['年'].iloc[0]])\
          & batter_HA['H/A'].isin(['HOME'])] + batter_HA.SLG[batter_HA['ID'].isin([match['球員ID'].iloc[0]]) \
            & batter_HA['year'].isin([match['年'].iloc[0]]) & batter_HA['H/A'].isin(['HOME'])]
OPS

In [ ]:
batter_HA.SLG[batter_HA['ID'].isin([match['球員ID'].iloc[0]]) & batter_HA['year'].isin([match['年'].iloc[0]])\
          & batter_HA['H/A'].isin(['HOME'])]

In [ ]:
#左右投
if match['球員ID'].iloc[0] ==